In [1]:
# load env with api keys https://stackoverflow.com/a/54028874
%load_ext dotenv
%dotenv ../etc/config.env

In [2]:
import sys
sys.path.append("../")

import pandas as pd

from desci_sense.parsers.base_parser import BaseParser
from desci_sense.configs import init_config
from desci_sense.schema.post import RefPost
from desci_sense.schema.templates import TEMPLATES, LABEL_TEMPLATE_MAP, DEFAULT_PREDICATE_LABEL, DISP_NAME_TEMPLATES_MAP

In [3]:
config = init_config(template_path="desci_sense/prompting/templates/p7.txt")
parser = BaseParser(config=config)

WARNING! headers is not default parameter.
                    headers was transferred to model_kwargs.
                    Please confirm that headers is what you intended.


In [4]:
# TEST_TWEET = "https://twitter.com/bingbrunton/status/1719789465739333972"
# result = parser.process_url(TEST_TWEET)
# result

In [4]:
result = {'post': RefPost(author='Bing Wen Brunton', content='My department at Univ Washington in Seattle is searching for a tenure-track assistant prof in "Quantitative Understanding of Collective Behavior" @UWBiology\n\nSee ad for more info about the search vision and to apply; happy to answer questions!\n\nhttps://apply.interfolio.com/130336 https://t.co/ydKZVuAeeY', url='https://twitter.com/bingbrunton/status/1719789465739333972', metadata={'communityNote': None, 'conversationID': '1719789465739333972', 'date': 'Wed Nov 01 18:52:15 +0000 2023', 'date_epoch': 1698864735, 'hashtags': [], 'likes': 128, 'mediaURLs': ['https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg'], 'media_extended': [{'altText': None, 'size': {'height': 1279, 'width': 3199}, 'thumbnail_url': 'https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg', 'type': 'image', 'url': 'https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg'}], 'possibly_sensitive': False, 'qrtURL': None, 'replies': 2, 'retweets': 74, 'text': 'My department at Univ Washington in Seattle is searching for a tenure-track assistant prof in "Quantitative Understanding of Collective Behavior" @UWBiology\n\nSee ad for more info about the search vision and to apply; happy to answer questions!\n\nhttps://apply.interfolio.com/130336 https://t.co/ydKZVuAeeY', 'tweetID': '1719789465739333972', 'tweetURL': 'https://twitter.com/bingbrunton/status/1719789465739333972', 'user_name': 'Bing Wen Brunton', 'user_profile_image_url': 'https://pbs.twimg.com/profile_images/1308602220179369984/3vv__cHD_normal.jpg', 'user_screen_name': 'bingbrunton'}, source_network='twitter', ref_urls=['https://apply.interfolio.com/130336']),
 'answer': {'reasoning': '[Reasoning Steps]\n\n1. The post contains a job announcement from the author\'s department at the University of Washington in Seattle.\n2. The author is searching for a tenure-track assistant professor in the field of "Quantitative Understanding of Collective Behavior".\n3. The author has included a link to the job application and is open to answering questions.\n\n[Candidate Tags]\n\n1. <job>\n2. <announce>\n3. <event> (only if the job announcement is an invitation to an event, not for a job listing)',
  'final_answer': '<job>',
  'single_tag': ['job'],
  'multi_tag': ['job']}}
result

{'post': RefPost(author='Bing Wen Brunton', content='My department at Univ Washington in Seattle is searching for a tenure-track assistant prof in "Quantitative Understanding of Collective Behavior" @UWBiology\n\nSee ad for more info about the search vision and to apply; happy to answer questions!\n\nhttps://apply.interfolio.com/130336 https://t.co/ydKZVuAeeY', url='https://twitter.com/bingbrunton/status/1719789465739333972', metadata={'communityNote': None, 'conversationID': '1719789465739333972', 'date': 'Wed Nov 01 18:52:15 +0000 2023', 'date_epoch': 1698864735, 'hashtags': [], 'likes': 128, 'mediaURLs': ['https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg'], 'media_extended': [{'altText': None, 'size': {'height': 1279, 'width': 3199}, 'thumbnail_url': 'https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg', 'type': 'image', 'url': 'https://pbs.twimg.com/media/F92_tj1asAAAoR-.jpg'}], 'possibly_sensitive': False, 'qrtURL': None, 'replies': 2, 'retweets': 74, 'text': 'My department at Univ Was

In [5]:
# placeholder for post in RDF triplet
POST_RDF = "post"

# display name for post for rendering in streamlit
POST_DISPLAY_NAME = "💬 Your post"

def create_triples_from_prediction(prediction):
    
    # TODO access to prediction attributes should be standardized
    # extract list of referenced links
    post: RefPost = prediction.get("post", None)
    if post:
        ref_links = post.ref_urls
    else:
        # the prediction didn't include a post (eg call to `process_text`)
        ref_links = []


    # extract predicted predicates
    predicted_predicates = prediction["answer"]["multi_tag"]
    
    # if no predicates were predicted, add the default predicate
    if len(predicted_predicates) == 0:
        predicted_predicates = [DEFAULT_PREDICATE_LABEL]

    # create table of all extracted triples
    rows = []
    for label in predicted_predicates:
        rows += [(POST_RDF, label ,link) for link in ref_links] 
    
    return rows

rows = create_triples_from_prediction(result)
rows

[('post', 'job', 'https://apply.interfolio.com/130336')]

In [7]:
rows += rows
rows

[('post', 'job', 'https://apply.interfolio.com/130336'),
 ('post', 'job', 'https://apply.interfolio.com/130336'),
 ('post', 'job', 'https://apply.interfolio.com/130336'),
 ('post', 'job', 'https://apply.interfolio.com/130336')]

In [8]:
df = pd.DataFrame(rows, columns=["Subject", "Predicate" ,"Object"])
df

,Subject,Predicate,Object
0,post,job,https://apply.interfolio.com/130336
1,post,job,https://apply.interfolio.com/130336
2,post,job,https://apply.interfolio.com/130336
3,post,job,https://apply.interfolio.com/130336


In [10]:
list(df.Predicate.unique())

['job']

In [12]:

rows = [('post', 'job', 'https://uchv.princeton.edu/postdoc-cog-sci'), ('post', 'event', 'https://uchv.princeton.edu/postdoc-cog-sci')]
df = pd.DataFrame(rows, columns=["Subject", "Predicate" ,"Object"])
df

,Subject,Predicate,Object
0,post,job,https://uchv.princeton.edu/postdoc-cog-sci
1,post,event,https://uchv.princeton.edu/postdoc-cog-sci


In [ ]:
df['predicate'] = df['predicate'].apply(lambda x: TEMPLATES[LABEL_TEMPLATE_MAP[x]]["display_name"])

In [13]:
DISP_NAME_TEMPLATES_MAP = {v["display_name"]: k for k,v in TEMPLATES.items()}
DISP_NAME_TEMPLATES_MAP

{'📢 announces-job': 'announcement-job',
 '📢 announces-research': 'Announcement-research',
 '🗓️ announces-event': 'Announcement-event',
 '🗣️ discusses': 'Discussion',
 '📝 quotes-from': 'Includes-quotation-from',
 '❔ ask-question-about': 'asks-question-about',
 '👌 recommends': 'recommends',
 '🧐 reviews': 'reviews',
 '⬛ links-to': 'links to',
 '🎧 listening-status': 'listening-consumption-status',
 '📑 reading-status': 'reading-consumption-status',
 '👀 watching-status': 'watching-consumption-status'}